# MWS Secondary Target Selection

## MWS_SPECIAL_RRLYR_SV

Maintainers:
  - Sergey Koposov (selection/catalog)
  - Andrew Cooper (secondary target code)
  
See `/project/projectdirs/desi/target/secondary/README` for output data model.

In [1]:
!pwd
!ls ../raw

/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/code
gaiarr_source_ids_pos_pm.fits


In [1]:
# Parameters

target_class   = 'MWS_SPECIAL_RRLYR_SV'
input_filename = 'gaiarr_source_ids_pos_pm.fits'

REF_CAT        = 'G'
REF_EPOCH      = 2015.5

colmap = {
    'ra':        'RA',
    'dec':       'DEC',
    'pmra':      'PMRA',
    'pmdec':     'PMDEC',
    'ref_epoch': 'REF_EPOCH',
    'source_id': 'REF_ID'
}

We include all the stars from Gaia that were classified as RR Lyrae with mean G magnitudes between 14 and 19. Specifically that includes

1. All the stars from Gaia vari_rrlyrae table
2. All the stars from vari_classifier_result with RR* class

The selection criteria are:

```
WITH x as (SELECT vari_classifier_result.source_id
        FROM gaia_dr2.vari_classifier_result
        WHERE vari_classifier_result.best_class_name::text ~~ 'RR%'::text
    UNION
    SELECT vari_rrlyrae.source_id
        FROM gaia_dr2.vari_rrlyrae)
SELECT g.* FROM gaia_dr2.gaia_source as g, x
       where g.source_id = x.source_id
       and phot_g_mean_mag between 14 and 19;
```

In [ ]:
from   importlib import reload
import sys
PYPATH = '/project/projectdirs/desi/external/mws/py/'
if PYPATH not in sys.path: sys.path.append(PYPATH)
import mwsecondary.mwsecondary as mws
reload(mws)

import os
import sys
import numpy as np

targets = mws.SecondaryTargets(target_class,input_filename,ref_cat=REF_CAT,ref_epoch=REF_EPOCH)
targets.print_info()
mask    = np.repeat(True,len(targets.data))
mask   &= targets.select_finite_range('dec',lower_limit=-30.0,verbose=True)
mask   &= targets.select_unique('source_id',verbose=True)

# Adjust to secondary target data model and write output
output = targets.get_output(mask,cols=colmap,write=True)
print(output[0:3])